In [ ]:
import asyncio
import hypercorn.asyncio
from hypercorn.config import Config
from urllib.parse import urlparse, unquote
import os

class SingleTCPHTTP2Server:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.connection = None

    async def process_request(self, scope, receive, send):

        #print("self = ",self)
        #print("scope = ",scope)
        #print("scope path = ",scope['path'] ) # /Data_files/B_10MB
        #print("receive = ",receive)
        #print("send = ",send)
        if scope['path'] == '/':
            await send({"type": "http.response.start", "status": 404, "headers": []})
            await send({"type": "http.response.body", "body": b"File not found"})
            await receive()
        else:
            file_path = '.'+scope['path']
            if os.path.isfile(file_path):
                await send({"type": "http.response.start", "status": 200, "headers": []})
                with open(file_path, 'rb') as file:
                    file_content = file.read()
                    await send({"type": "http.response.body", "body": file_content})
                await receive()
            else:
                await send({"type": "http.response.start", "status": 404, "headers": []})
                await send({"type": "http.response.body", "body": b"Filessss not found"})
                await receive()

    async def run_server(self):
        config = Config()
        config.bind = [f"{self.host}:{self.port}"]
        print("=",config.bind)
        config.http2 = True
        await hypercorn.asyncio.serve(self.process_request, config)

# Run the server in an asyncio event loop
async def main():
    server = SingleTCPHTTP2Server('', 8895)
    print('Server started on port 8895')
    await server.run_server()

await main()

[2024-03-03 00:06:07 -0500] [28069] [WARNING] ASGI Framework Lifespan error, continuing without Lifespan support
[2024-03-03 00:06:07 -0500] [28069] [INFO] Running on http://0.0.0.0:8895 (CTRL + C to quit)


Server started on port 8895
= [':8895']
